### Compare different formulation of the Peierls and Yield stress.

In the note book, I compared the values from yield criterias and Peierls rheologies.

In [ ]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))


import shilofue.Rheology as Rheology


# define global parameters
g = 9.81
rho_m = 3300.0 # mantle density

#### Gerya et al 2002

This paper introduces a wet byerlee law.
The profile of yield stress predepends on a profile of the factor of the core fluid pressure ($\lambda$)

The profile of the factor of the core fluid pressure ($\lambda$) is generated from a linear relation.
The factor at the surface is assumed to be 0.4, while the factor at 10 km depth is either dry or lithostatic.

The condition for a lithostatic condition at 10 km depth is from Etheridge etal : $\lambda = 0.99$.

In [ ]:
def Gerya_2002_lambda(z, condition):
    '''
    Expression of pore fluid pressure factor lambda
    '''
    assert(condition in [0, 1])
    a = 0.4 # hydrostatic pore fluid pressure at the surface
    b = 0.0 # dry at z = 10 km, condition = 0
    if condition == 1:
        # lithostatic pore fluid pressure at z = 10 km
        b = 0.99
    _lambda = (a * (1e4 - z) + b * z) / 10000

    return _lambda

z = 5e3
_lambda_dry_5km = Gerya_2002_lambda(5e3, 0)
print("lambda at 5km assuming dry condition at 10 km: ", _lambda_dry_5km)

_lambda_wet_5km = Gerya_2002_lambda(5e3, 1)
print("lambda at 5km assuming lithostatic pore fluid pressure condition at 10 km: ", _lambda_wet_5km)

##### Relation between variables

The shear stress ($\tau$) is half the value of the differential stress ($\sigma_d$).

The differential stress ($\sigma_d$) is twice the second invariant of stress ($\sigma_{ii}$)

The effective viscosity is computed from the second invariants.

$$\tau = 1/2 \sigma_d$$
$$\sigma_d = 2 \sigma_{ii}$$
$$\eta_{eff} = \frac{\sigma_{ii}}{2 \dot{\epsilon_{ii}}}$$

So if we formulate yield stress from the shear stress, the factor of 2 should be in the deliminator.
If we formulate yield- stress from the differential stress, then there should be a factor of 4 ?

In [ ]:
zs_t2002 = np.linspace(0.0, 10e3, 1000)
Ps = zs_t2002 * rho_m * g
lambdas_0 = Gerya_2002_lambda(zs_t2002, 0)
taus_0 = Rheology.Byerlee(Ps, _lambda=lambdas_0)
lambdas_1 = Gerya_2002_lambda(zs_t2002, 1)
taus_1 = Rheology.Byerlee(Ps, _lambda=lambdas_1)

n_z = np.argmax(taus_1)
tau_1_max = taus_1[n_z]
z_1_max = zs_t2002[n_z]
print("For 1, the maximum shear stress %.2f MPa is reached at depth %.2f km." % (tau_1_max/1e6, z_1_max/1e3))

fig = plt.figure(tight_layout=True, figsize=(10, 5))
gs = gridspec.GridSpec(1, 2)

# shear stress vs depth
ax = fig.add_subplot(gs[0, 0])
ax.plot(taus_0/1e6, zs_t2002/1e3, label="Dry condition at 10 km")
ax.plot(taus_1/1e6, zs_t2002/1e3, label="Lithostatic pore fluid pressure at 10 km")
ax.set_xlabel("Shear Stress (MPa)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()

# viscosity vs depth
strain_rate = 1e-15  # second invariant
etas_0_t2002 = taus_0 / 2.0 / strain_rate
etas_1_t2002 = taus_1 / 2.0 / strain_rate
ax = fig.add_subplot(gs[0, 1])
ax.semilogx(etas_0_t2002, zs_t2002/1e3, label="Dry condition at 10 km")
ax.semilogx(etas_1_t2002, zs_t2002/1e3, label="Lithostatic pore fluid pressure at 10 km")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()
print("For 1, the maximum effective viscosity is %.2e Pa (strain rate %.2e)" % (etas_1_t2002[n_z], strain_rate))

#### The stress limiter approach

In [ ]:
tau_y = 500e6 # 500 Mpa
strain_rate_y = 1e-15
ny = 10
strain_rate = 1e-15
tau = Rheology.PeierlsStressLimiter(tau_y, strain_rate_y, ny, strain_rate)
eta_y = tau / strain_rate
print("At strain rate = %.2e, the yield stress is %.2e and the yield viscosity is %.2e" % (strain_rate, tau, eta_y))
strain_rate = 1e-14
tau = Rheology.PeierlsStressLimiter(tau_y, strain_rate_y, ny, strain_rate)
eta_y = tau / strain_rate
print("At strain rate = %.2e, the yield stress is %.2e and the yield viscosity is %.2e" % (strain_rate, tau, eta_y))
strain_rate = 1e-13
tau = Rheology.PeierlsStressLimiter(tau_y, strain_rate_y, ny, strain_rate)
eta_y = tau / strain_rate
print("At strain rate = %.2e, the yield stress is %.2e and the yield viscosity is %.2e" % (strain_rate, tau, eta_y))

In [ ]:
zs_g2014 = np.linspace(0.0, 10e3, 1000)
Ps = zs_g2014 * rho_m * g
tau_0 = 2e6
tau_y = 1000e6
friction = 0.2
taus_g2014 = Rheology.CoulumbYielding(Ps, tau_0, friction)

fig = plt.figure(tight_layout=True, figsize=(15, 5))
gs = gridspec.GridSpec(1, 3)

# shear stress vs depth
ax = fig.add_subplot(gs[0, 0])
ax.plot(taus_g2014/1e6, zs_g2014/1e3, label="Garel 2014")
ax.set_xlabel("Shear Stress (MPa)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()

# viscosity vs depth
strain_rate = 1e-15  # second invariant
etas_g2014 = taus_g2014 / 2.0 / strain_rate
ax = fig.add_subplot(gs[0, 1])
ax.semilogx(etas_g2014, zs_g2014/1e3, label="Garel 2014")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.set_title("strain rate = %.2e" % strain_rate)

# viscosity vs depth
strain_rate = 1e-13  # second invariant
etas_g2014 = taus_g2014 / 2.0 / strain_rate
ax = fig.add_subplot(gs[0, 2])
ax.semilogx(etas_g2014, zs_g2014/1e3, label="Garel 2014")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.set_title("strain rate = %.2e" % strain_rate)